<a href="https://colab.research.google.com/github/KaydeeJR/logistics-optimization-causal-inference/blob/Main/notebooks/gokada_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/Datasets/GokadaData/
driver_locations = "driver_locations_during_request.csv"
feat_eng_file = "gokada_feat_eng.csv"

/content/drive/MyDrive/Datasets/GokadaData


In [2]:
import pandas as pd
df_feat_eng = pd.read_csv(feat_eng_file)

## Missing values

In [3]:
df_feat_eng.isnull().sum() * 100 / len(df_feat_eng)

Unnamed: 0            0.000000
Trip ID               0.000000
Trip Origin           0.000000
Trip Destination      0.000000
Trip Start Time       0.696779
Trip End Time         0.000000
order_id              0.000000
driver_id             0.000000
driver_action         0.000000
lat                   0.000000
lng                   0.000000
created_at          100.000000
updated_at          100.000000
start_hour            0.696779
end_hour              0.000000
trip_distance_km      0.000000
trip_time             0.696779
Start Date            0.696779
rainy                 0.000000
holiday               0.000000
weekend               0.000000
dtype: float64

*Empty values resulted from trip start time*

In [4]:
col_to_drop = ['created_at','updated_at','Unnamed: 0', 'Start Date', 'holiday']
new_df = df_feat_eng.drop(col_to_drop, axis=1)

In [5]:
df_missing_val = new_df.dropna()

In [6]:
df_missing_val.isnull().sum()

Trip ID             0
Trip Origin         0
Trip Destination    0
Trip Start Time     0
Trip End Time       0
order_id            0
driver_id           0
driver_action       0
lat                 0
lng                 0
start_hour          0
end_hour            0
trip_distance_km    0
trip_time           0
rainy               0
weekend             0
dtype: int64

In [7]:
df_missing_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1546886 entries, 0 to 1557739
Data columns (total 16 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Trip ID           1546886 non-null  int64  
 1   Trip Origin       1546886 non-null  object 
 2   Trip Destination  1546886 non-null  object 
 3   Trip Start Time   1546886 non-null  object 
 4   Trip End Time     1546886 non-null  object 
 5   order_id          1546886 non-null  int64  
 6   driver_id         1546886 non-null  int64  
 7   driver_action     1546886 non-null  object 
 8   lat               1546886 non-null  float64
 9   lng               1546886 non-null  float64
 10  start_hour        1546886 non-null  float64
 11  end_hour          1546886 non-null  int64  
 12  trip_distance_km  1546886 non-null  float64
 13  trip_time         1546886 non-null  object 
 14  rainy             1546886 non-null  bool   
 15  weekend           1546886 non-null  bool   
dtype

In [8]:
df_missing_val.describe()

,Trip ID,order_id,driver_id,lat,lng,start_hour,end_hour,trip_distance_km
count,1.546886e+06,1.546886e+06,1.546886e+06,1.546886e+06,1.546886e+06,1.546886e+06,1.546886e+06,1.546886e+06
mean,1.319561e+06,4.192284e+05,2.341530e+05,6.536097e+00,3.378828e+00,1.345592e+01,1.440968e+01,1.182662e+01
std,3.195611e+04,1.442346e+04,2.270416e+04,5.991502e-02,5.328545e-02,3.305160e+00,3.371022e+00,8.410192e+00
min,1.259089e+06,3.920010e+05,1.219810e+05,6.409333e+00,3.076561e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.293251e+06,4.069540e+05,2.429970e+05,6.498711e+00,3.348846e+00,1.100000e+01,1.200000e+01,5.031020e+00
50%,1.324634e+06,4.217350e+05,2.435890e+05,6.544250e+00,3.363504e+00,1.400000e+01,1.500000e+01,1.014846e+01
75%,1.340384e+06,4.293130e+05,2.440560e+05,6.593231e+00,3.385136e+00,1.600000e+01,1.700000e+01,1.707343e+01
max,1.570716e+06,5.179480e+05,2.478770e+05,7.702536e+00,8.515414e+00,2.300000e+01,2.300000e+01,8.499244e+01


## Outliers

In [9]:
!pip install seaborn
import seaborn as sns

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
numeric_cols = df_missing_val.select_dtypes(include=['float64','int64']) 
column_list = list(numeric_cols.columns)

In [11]:
!pip install sklearn
import sklearn
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=c8402b99cd8b26572e5c50a30380027bb6f89815778836b556fd0924ef99311f
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [12]:
# Detecting and removing outliers using IQR
Q1 = df_missing_val[column_list].quantile(0.25)
Q3 = df_missing_val[column_list].quantile(0.75)
IQR = Q3 - Q1

# Return a boolean array of the rows with (any) non-outlier column values
condition = ~((df_missing_val[column_list]< (Q1 - 1.5 * IQR)) | (df_missing_val[column_list] > (Q3 + 1.5 * IQR))).any(axis=1)

# Filter our dataframe based on condition
clean_df = df_missing_val[condition]

In [13]:
clean_df.describe()

,Trip ID,order_id,driver_id,lat,lng,start_hour,end_hour,trip_distance_km
count,9.807470e+05,980747.000000,980747.000000,980747.000000,980747.000000,980747.000000,980747.000000,980747.000000
mean,1.318879e+06,418942.430456,243882.206206,6.549530,3.363414,13.476043,14.439124,11.651529
std,3.150089e+04,14241.921532,758.286486,0.050183,0.023268,3.309323,3.357869,7.934069
min,1.259098e+06,392005.000000,241760.000000,6.409333,3.294447,5.000000,5.000000,0.000000
25%,1.292260e+06,406510.000000,243395.000000,6.517622,3.347414,11.000000,12.000000,5.050384
50%,1.324692e+06,421766.000000,243767.000000,6.544484,3.363415,14.000000,15.000000,10.148461
75%,1.339620e+06,428892.000000,244108.000000,6.594491,3.375737,16.000000,17.000000,16.990991
max,1.386520e+06,448616.000000,245644.000000,6.728332,3.439562,23.000000,23.000000,35.128115


## One Hot Encoding

In [14]:
cat_columns = ['rainy','weekend']

encoded_df = pd.get_dummies(clean_df, columns = cat_columns)

In [15]:
encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 980747 entries, 10 to 1557591
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Trip ID           980747 non-null  int64  
 1   Trip Origin       980747 non-null  object 
 2   Trip Destination  980747 non-null  object 
 3   Trip Start Time   980747 non-null  object 
 4   Trip End Time     980747 non-null  object 
 5   order_id          980747 non-null  int64  
 6   driver_id         980747 non-null  int64  
 7   driver_action     980747 non-null  object 
 8   lat               980747 non-null  float64
 9   lng               980747 non-null  float64
 10  start_hour        980747 non-null  float64
 11  end_hour          980747 non-null  int64  
 12  trip_distance_km  980747 non-null  float64
 13  trip_time         980747 non-null  object 
 14  rainy_False       980747 non-null  uint8  
 15  rainy_True        980747 non-null  uint8  
 16  weekend_False     

## Splitting dataset

In [ ]:
!pip install sklearn
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(encoded_df, y, test_size=0.20, shuffle=False)